In [1]:
import torch
print(torch.__version__)

2.2.2


In [2]:
import os
os.environ["MKL_THREADING_LAYER"] = "TBB"

In [3]:
import ctgan
dir(ctgan)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


['CTGAN',
 'TVAE',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'data_sampler',
 'data_transformer',
 'demo',
 'load_demo',
 'synthesizers']

In [4]:
import pandas as pd
import numpy as np
from ctgan import CTGAN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import ks_2samp, chi2_contingency
import time

In [5]:
df = pd.read_csv("com_salary.csv")

In [6]:
df.head()

,Name,OttoneuID,FG MajorLeagueID,FG MinorLeagueID,MLB Org,Position(s),Avg Salary,Median Salary,Min Salary,Max Salary,...,POS,ADP,rPTS,PTS,aPOS,Dollars,Adjusted,Cost,PlayerId,value
0,Juan Soto,23717,20123.0,sa906282,NYY,OF,60.31,60.0,34,84,...,OF/DH,10.57,1132.535865,53.429347,21.885623,76.314969,112.280793,NaN,20123,0.000000
1,Mookie Betts,18276,13611.0,sa597889,LAD,2B/SS/OF,58.35,58.0,38,87,...,2B/SS/OF,5.03,1036.172159,40.095221,14.934803,56.030024,82.308998,NaN,13611,0.000000
2,Shohei Ohtani,33600,19755.0,NaN,LAD,Util/SP,56.13,56.0,32,88,...,P/DH,12.73,1038.276653,40.386426,12.178202,53.564628,75.049219,54.0,19755,-0.435372
3,Aaron Judge,18312,15640.0,sa549847,NYY,OF,54.17,54.5,33,70,...,OF/DH,11.16,1066.255227,44.257902,21.885623,67.143525,93.070498,50.0,15640,17.143525
4,Freddie Freeman,5680,5361.0,sa390291,LAD,1B,50.76,51.0,27,74,...,1B,8.81,1062.235872,43.701733,13.809086,58.510819,81.663561,47.0,5361,11.510819



# Assuming `orig_df` is your original DataFrame with the actual data
# Example DataFrame (replace with your actual DataFrame)
orig_df = pd.DataFrame({
    'OttoneuID': [1, 2, 3, 4, 5],
    'FG MajorLeagueID': [1001, 1002, 1003, 1004, 1005],
    'Avg Salary': [10, 20, 30, 40, 50],
    'Median Salary': [15, 25, 35, 45, 55],
    'Min Salary': [5, 10, 15, 20, 25],
    'Max Salary': [20, 30, 40, 50, 60],
    'Last 10': [7, 8, 9, 10, 11],
    'Roster%': [60, 70, 80, 90, 100],
    'ADP': [1, 2, 3, 4, 5],
    'rPTS': [100, 200, 300, 400, 500],
    'PTS': [150, 250, 350, 450, 550],
    'aPOS': [1, 2, 3, 4, 5],
    'Dollars': [10, 20, 30, 40, 50],
    'Adjusted': [5, 10, 15, 20, 25],
    'Cost': [2, 4, 6, 8, 10],
    'value': [8, 16, 24, 32, 40],
    'Name': ['Player1', 'Player2', 'Player3', 'Player4', 'Player5'],
    'FG MinorLeagueID': ['ID1', 'ID2', 'ID3', 'ID4', 'ID5'],
    'MLB Org': ['TeamA', 'TeamB', 'TeamC', 'TeamD', 'TeamE'],
    'Position(s)': ['Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5'],
    'Team': ['Team1', 'Team2', 'Team3', 'Team4', 'Team5'],
    'POS': ['POS1', 'POS2', 'POS3', 'POS4', 'POS5'],
    'PlayerId': ['P1', 'P2', 'P3', 'P4', 'P5']
})

In [7]:
orig_df = pd.DataFrame(df)
orig_df.head()

,Name,OttoneuID,FG MajorLeagueID,FG MinorLeagueID,MLB Org,Position(s),Avg Salary,Median Salary,Min Salary,Max Salary,...,POS,ADP,rPTS,PTS,aPOS,Dollars,Adjusted,Cost,PlayerId,value
0,Juan Soto,23717,20123.0,sa906282,NYY,OF,60.31,60.0,34,84,...,OF/DH,10.57,1132.535865,53.429347,21.885623,76.314969,112.280793,NaN,20123,0.000000
1,Mookie Betts,18276,13611.0,sa597889,LAD,2B/SS/OF,58.35,58.0,38,87,...,2B/SS/OF,5.03,1036.172159,40.095221,14.934803,56.030024,82.308998,NaN,13611,0.000000
2,Shohei Ohtani,33600,19755.0,NaN,LAD,Util/SP,56.13,56.0,32,88,...,P/DH,12.73,1038.276653,40.386426,12.178202,53.564628,75.049219,54.0,19755,-0.435372
3,Aaron Judge,18312,15640.0,sa549847,NYY,OF,54.17,54.5,33,70,...,OF/DH,11.16,1066.255227,44.257902,21.885623,67.143525,93.070498,50.0,15640,17.143525
4,Freddie Freeman,5680,5361.0,sa390291,LAD,1B,50.76,51.0,27,74,...,1B,8.81,1062.235872,43.701733,13.809086,58.510819,81.663561,47.0,5361,11.510819


In [8]:
# Selecting numeric and categorical features
numeric_features = orig_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = orig_df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

Numeric features: ['OttoneuID', 'FG MajorLeagueID', 'Avg Salary', 'Median Salary', 'Min Salary', 'Max Salary', 'Last 10', 'Roster%', 'ADP', 'rPTS', 'PTS', 'aPOS', 'Dollars', 'Adjusted', 'Cost', 'value']
Categorical features: ['Name', 'FG MinorLeagueID', 'MLB Org', 'Position(s)', 'Team', 'POS', 'PlayerId']


In [9]:
# Define transformers
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [10]:
# Fit and transform the data
prepared_data = preprocessor.fit_transform(orig_df)

# Check if the data is sparse and convert to dense if necessary
if hasattr(prepared_data, "toarray"):
    prepared_data = prepared_data.toarray()

# Check the shape of the prepared data
print(f"Shape of prepared_data: {prepared_data.shape}")

Shape of prepared_data: (909, 2901)


In [11]:
# Get the column names from the transformers
numeric_column_names = numeric_features
categorical_column_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)

# Combine all column names
all_column_names = numeric_column_names + list(categorical_column_names)

# Verify the lengths match
print(f"Number of columns in prepared_data: {prepared_data.shape[1]}")
print(f"Number of column names: {len(all_column_names)}")

Number of columns in prepared_data: 2901
Number of column names: 2901


In [12]:
# Ensure the lengths match before creating the DataFrame
if prepared_data.shape[1] == len(all_column_names):
    prepared_df = pd.DataFrame(prepared_data, columns=all_column_names)
    print(prepared_df.head())
else:
    raise ValueError("The number of columns in the prepared data does not match the number of column names.")

   OttoneuID  FG MajorLeagueID  Avg Salary  Median Salary  Min Salary  \
0  -0.209873          0.247488    6.930061       6.916505    8.003330   
1  -0.753909         -0.833026    6.678714       6.660718    9.023420   
2   0.778310          0.186427    6.394025       6.404932    7.493285   
3  -0.750309         -0.496361    6.142677       6.213092    7.748307   
4  -2.013360         -2.201921    5.705385       5.765466    6.218172   

   Max Salary   Last 10   Roster%       ADP      rPTS  ...  \
0    4.873362  5.999952  1.375820 -1.621671  3.108094  ...   
1    5.083293  6.062393  1.375820 -1.640549  2.692647  ...   
2    5.153270  5.302691  1.343176 -1.614311  2.701720  ...   
3    3.893686  5.042520  1.449258 -1.619661  2.822342  ...   
4    4.173594  4.397294  1.392143 -1.627668  2.805014  ...   

   PlayerId_sa3020241  PlayerId_sa3022654  PlayerId_sa3022882  \
0                 0.0                 0.0                 0.0   
1                 0.0                 0.0                 

In [13]:
# Initialize the CTGAN model
ctgan = CTGAN(epochs=2)  # Adjust the number of epochs as needed

# Train the CTGAN model
ctgan.fit(prepared_df, categorical_column_names)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible worka

KeyboardInterrupt: 

In [ ]:
def generate_synthetic_data(file_path, additional_rows, epochs=5, output_file='synthetic_data_ctgan.csv'):
    # Load the original dataset
    original_df = pd.read_csv(file_path)
    
    # Selecting numeric and categorical features
    numeric_features = original_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = original_df.select_dtypes(include=['object', 'category']).columns.tolist()

    # Define transformers
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine transformers into a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Fit and transform the data
    prepared_data = preprocessor.fit_transform(original_df)

    # Check if the data is sparse and convert to dense if necessary
    if hasattr(prepared_data, "toarray"):
        prepared_data = prepared_data.toarray()

    # Get the column names from the transformers
    numeric_column_names = numeric_features
    categorical_column_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)

    # Combine all column names
    all_column_names = numeric_column_names + list(categorical_column_names)

    # Ensure the lengths match before creating the DataFrame
    if prepared_data.shape[1] == len(all_column_names):
        prepared_df = pd.DataFrame(prepared_data, columns=all_column_names)
    else:
        raise ValueError("Mismatch between prepared data columns and column names.")

    # Initialize the CTGAN model
    ctgan = CTGAN(epochs=epochs)

    # Train the CTGAN model
    ctgan.fit(prepared_df, categorical_column_names)

    # Specify the number of synthetic samples to generate
    num_samples = len(original_df) + additional_rows

    # Generate synthetic data and measure time taken
    start_time = time.time()
    synthetic_data = ctgan.sample(num_samples)
    elapsed_time = time.time() - start_time

    # Convert synthetic data to a pandas DataFrame with appropriate column names
    synthetic_df = pd.DataFrame(synthetic_data, columns=all_column_names)

    # Save the synthetic data to a CSV file
    synthetic_df.to_csv(output_file, index=False)
    print(f"Synthetic data written to '{output_file}' successfully with {num_samples} samples.")
    print(f"Synthetic data generation completed in {elapsed_time:.2f} seconds.")
    
    return synthetic_df


In [ ]:
# Define Evaluation Functions
def evaluate_fidelity(real_data, synthetic_data, continuous_columns, categorical_columns):
    ks_results = {col: ks_2samp(real_data[col], synthetic_data[col]).statistic for col in continuous_columns}
    chi_squared_results = {col: chi2_contingency(pd.crosstab(real_data[col], synthetic_data[col]))[:2] for col in categorical_columns}
    return {'KS Test': ks_results, 'Chi-Squared Test': chi_squared_results}

In [ ]:
def evaluate_predictive_performance(real_data, synthetic_data, target_column, test_size=0.3, random_state=42):
    X_real = real_data.drop(columns=[target_column])
    y_real = real_data[target_column]
    X_synthetic = synthetic_data.drop(columns=[target_column])
    y_synthetic = synthetic_data[target_column]
    model = RandomForestClassifier(n_estimators=100, random_state=random_state)
    model.fit(X_synthetic, y_synthetic)
    predictions = model.predict(X_real)
    return {
        'Accuracy': accuracy_score(y_real, predictions),
        'ROC AUC': roc_auc_score(y_real, model.predict_proba(X_real)[:, 1]),
        'F1 Score': f1_score(y_real, predictions)
    }

In [ ]:
def informativeness_test(real_data, synthetic_data, test_size=0.3, random_state=42):
    real_data['is_real'] = 1
    synthetic_data['is_real'] = 0
    combined_data = pd.concat([real_data, synthetic_data], ignore_index=True)
    X = combined_data.drop(columns=['is_real'])
    y = combined_data['is_real']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    classifier = RandomForestClassifier(n_estimators=100, random_state=random_state)
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    return {
        'Accuracy': accuracy_score(y_test, predictions),
        'ROC AUC': roc_auc_score(y_test, classifier.predict_proba(X_test)[:, 1]),
        'F1 Score': f1_score(y_test, predictions)
    }

In [ ]:
# Evaluate the synthetic data
def evaluate_synthetic_data(original_df, synthetic_df, target_column):
    # Identify columns to apply metrics
    continuous_columns = original_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_columns = original_df.select_dtypes(include=['object']).columns.tolist()

    # Evaluate fidelity
    fidelity_results = evaluate_fidelity(original_df, synthetic_df, continuous_columns, categorical_columns)
    print("Fidelity Results:", fidelity_results)

    # Evaluate predictive performance
    if target_column in original_df.columns:
        predictive_performance_results = evaluate_predictive_performance(original_df, synthetic_df, target_column)
        print("Predictive Performance Results:", predictive_performance_results)
    else:
        print(f"Error: Target column '{target_column}' does not exist in the DataFrame.")

    # Evaluate informativeness
    informativeness_results = informativeness_test(original_df, synthetic_df)
    print("Informativeness Results:", informativeness_results)

In [ ]:
# Example usage of the function
if __name__ == "__main__":
    file_path = 'path_to_your_data.csv'  # Update this path to your dataset
    additional_rows = int(input("Enter the number of additional synthetic data samples to generate: "))
    epochs = int(input("Enter the number of epochs for training the CTGAN model: "))
    output_file = input("Enter the name of the output file (with .csv extension): ")
    
    synthetic_df = generate_synthetic_data(file_path, additional_rows, epochs=epochs, output_file=output_file)

    # Print some outputs to verify
    print("Generated synthetic data shape:", synthetic_df.shape)
    print("Sample of synthetic data:")
    print(synthetic_df.head())

In [ ]:
if __name__ == "__main__":
    target_column = 'your_target_column'  # Replace with your actual target column name
    evaluate_synthetic_data(original_df, synthetic_df, target_column)